In [6]:
!pip install transformers
!pip install scipy
!conda install pytorch torchvision -c pytorch
# Runtime might need restarting after this installation.
# !pip install torch torchvision 
!pip install twitscrape

# twitscrape is a package I created as an aside to this project to further bolster my learning (https://github.com/christywastaken/twitscrape, https://pypi.org/project/twitscrape). 
# I'm sure there are more effective ways of collecting the tweet data used in this project. If you are looking to repeat this data analysis I would reccomend looking for another package such as snscrape.


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Solving environment: done

# All requested packages already installed.


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import sqlite3
from twitscrape.twitter_scraper import TwitterGeolocationScraper 
# twitscrape is a package I built and distributed as an aside learning experiment to this project. I plan to work on it further so the import method might change from the above.
# https://github.com/christywastaken/twitscrape 
# https://pypi.org/project/twitscrape/ 

newcastle_latitude = 54.975029
newcastle_longitude = -1.612477
search_radius_km = 15.0
summer_solstice_week_scraper = TwitterGeolocationScraper(start_date='2022-06-20', end_date='2022-06-28', latitude=newcastle_latitude, longitude=newcastle_longitude, radius=search_radius_km, filter_links=True, filter_replies=True, is_headless=True)
summer_solstice_week_df = summer_solstice_week_scraper.run()
# Creates dataframe of all the tweets in Newcastle-Upon-Tyne for a 15km radius the week around the summer solstice 2022. Links and replies filtered out. 

#Store DF to SQL DB
conn = sqlite3.connect('twitter_sentiment_analysis.db')
summer_solstice_week_df.to_sql('summer_solstice_week', conn, if_exists='replace', index=False)
conn.close()



-- TwitterGeolocationScraper running. This may take a minute to update the webdriver. --
Page loading not complete
remaining rate limit: 249 | tweet_df length: 20
remaining rate limit: 247 | tweet_df length: 20
remaining rate limit: 246 | tweet_df length: 20
remaining rate limit: 245 | tweet_df length: 20
remaining rate limit: 244 | tweet_df length: 20
remaining rate limit: 243 | tweet_df length: 20
remaining rate limit: 242 | tweet_df length: 20
remaining rate limit: 241 | tweet_df length: 20
remaining rate limit: 240 | tweet_df length: 20
remaining rate limit: 239 | tweet_df length: 20
remaining rate limit: 238 | tweet_df length: 20
remaining rate limit: 237 | tweet_df length: 20
remaining rate limit: 236 | tweet_df length: 20
remaining rate limit: 235 | tweet_df length: 20
remaining rate limit: 234 | tweet_df length: 20
remaining rate limit: 233 | tweet_df length: 20
remaining rate limit: 232 | tweet_df length: 20
remaining rate limit: 231 | tweet_df length: 20
remaining rate limit:

In [12]:
import pandas as pd
import re

def clean_tweets(tweet_df: pd.DataFrame) -> pd.DataFrame:
    cleaned_tweet_df = tweet_df.copy()
    for index, row in cleaned_tweet_df.iterrows():
        tweet_text = row['tweet_text']
        clean_tweet_text = re.sub("@[A-Za-z0-9_]+","", tweet_text)
        #TODO: Should I replace @mentions with a name like 'Daniel' to help the sentiment analysis? Or should I include @mentions as original? test this.
        clean_tweet_text = re.sub("#[A-Za-z0-9_]+","", clean_tweet_text)
        clean_tweet_text = clean_tweet_text.lower()
        print(f'\ntweet: {tweet_text}\nclean: {clean_tweet_text}\n')
        
        cleaned_tweet_df.at[index, 'cleaned_tweet_text'] = clean_tweet_text
    
    return cleaned_tweet_df



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 2.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [36]:
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import numpy as np

#TODO: Check licensing and citing for https://github.com/cardiffnlp/timelms 
#TODO: consider saving model to local.

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


def analyse_sentiment(tweet_df: pd.DataFrame) -> pd.DataFrame:
    tweet_df_copy = tweet_df.copy()
    MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    config = AutoConfig.from_pretrained(MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    # model.save_pretrained(MODEL) #TODO: consider saving to local.
    for index, row in tweet_df_copy.iterrows():
        try:
            tweet_text = row['tweet_text']
            tweet_text = preprocess(tweet_text)
            encoded_input = tokenizer(tweet_text, return_tensors='pt')
            output = model(**encoded_input)
            scores = output[0][0].detach().numpy()
            scores = softmax(scores)
            scores = np.round(scores.astype(float), 4)
            sentiment_dict = {config.id2label[i]: score for i, score in enumerate(scores)}
            for column, value in sentiment_dict.items():
                tweet_df_copy.at[index, column] = value
            # print(f'tweet: {tweet_text} | senti: {sentiment_dict}')
        except Exception as err:
            print(f'Error: {err}')
    return tweet_df_copy



In [37]:
import sqlite3
import pandas as pd

def analyse_sentiment_and_update_sql_table(table_name: str):
    conn = sqlite3.connect('twitter_sentiment_analysis.db')
    sql_query = f"SELECT * FROM {table_name}"
    tweet_df = pd.read_sql(sql_query, conn)
    sentiment_tweet_df = analyse_sentiment(tweet_df)
    sentiment_tweet_df.to_sql(table_name, con=conn, if_exists='replace', index=False)
    conn.close()



In [38]:

analyse_sentiment_and_update_sql_table('summer_solstice_week')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
